In [1]:
from sklearn import preprocessing

In [2]:
def removeColumnsFromList(df, columnsToRemove):
    """
    Return a list of columns names excluding the names in the list 
    `columnsToKeep`.
    
    Args:
        df: pandas.core.frame.DataFrame
            The DataFrame used to produce the list of column names. 
        
        columnsToRemove: iterable
            An iterable object that has the names as elements that
            will be excluded from the returned list.
    Returns:
        list: The aforementioned column names.
    """
    columns = df.columns.tolist()
    for column in columnsToRemove:
        columns.remove(column)
        
    return columns

<p>We could easily reproduce these type of plots <a href="https://www.statsmodels.org/dev/examples/notebooks/generated/regression_plots.html">very closely</a>, with <a href="http://www.statsmodels.org/dev/examples/notebooks/generated/regression_diagnostics.html">additional regression diagnostics</a>, using the <code>statsmodels</code> library, however I would like to have more control, so I decided to plot most of them manually using <code>matplotlib</code>.

In [3]:
def createResidualPlots(X, Y, width=7, height=3):
    """
    This function returns various residual plots for the fitted model.
    
    For simple linear regressions, the first two plots are plots of the 
    residuals and the square root of the absolute standardized residuals
    vs the predictor. For the multiple regression fit, we instead plots 
    the residuals and the square root of the absolute standardized 
    residuals vs the fitted values. The third plot is a QQ plot of the
    quantiles of the standardized residuals vs the quantiles of the 
    normal distribution, and a 45-degree reference line is also plotted 
    for comparison (see also 
    https://seankross.com/2016/02/29/A-Q-Q-Plot-Dissection-Kit.html). The 
    final plot is a leverage plot of the standardized residuals.
    
    Args:
        X: pandas.core.frame.DataFrame
            The DataFrame should hold the data of independent variables
            (including a column for the 'Intercept' set equal to one).
            Each row in the DataFrame represents an individual sample 
            point, with the successive columns corresponding to the 
            independent variables and their specific values for that 
            sample point. 
        Y: pandas.core.frame.DataFrame or pandas.core.series.Series
            This should be a pandas Series of DataFrame of one column,
            which holds the data of the dependent variable.
        width: float, default 15
            The width of each subplot.
        height: float, default 4
            The height of each subplot.
    """ 
    columns_dependent_var = X.columns.values.tolist()
    columns_dependent_var.remove('Intercept')
    assert len(columns_dependent_var) >= 1, f'columns_dependent_var = {columns_dependent_var}'
    approach = 'simple' if len(columns_dependent_var) == 1 else 'multivariable'
    
    Y_hat = fitted.predict(X)
    residual = np.squeeze(Y.to_numpy()) - Y_hat
    standardized_residual = preprocessing.scale(residual)
    
    X_np = X.to_numpy()
    H = X_np @ np.linalg.inv(X_np.transpose() @ X_np) @ X_np.transpose()
    leverage = H.diagonal() 
    
    numberOfSubplots = 4
    fig, axes = plt.subplots(numberOfSubplots, 1, constrained_layout=True, figsize=(width, height*numberOfSubplots))
    if approach == 'simple':
        descriptive = columns_dependent_var[0]
        X_plot = X[descriptive]
    else:
        descriptive = 'fitted values'
        X_plot = Y_hat
    
    _ = axes[0].scatter(x=X_plot, y=residual)
    _ = axes[0].set_xlabel(descriptive)
    _ = axes[0].set_ylabel('residuals')
    _ = axes[0].set_title(f'residual plot for the linear regression')
    
    _ = axes[1].scatter(x=X_plot, y=np.absolute(standardized_residual)**0.5)
    _ = axes[1].set_xlabel(descriptive)
    _ = axes[1].set_ylabel(r'$\sqrt{\left|\mathrm{standardized \,\,\, residuals}\right|}$')
    _ = axes[1].set_title(r'$\sqrt{\left|\mathrm{standardized \,\,\, residuals}\right|}$ for the linear regression')
    
    _ = sm.qqplot(standardized_residual, line='45', c=None, ax=axes[2])
    _ = axes[2].set_xlabel('theoretical qunatiles')
    _ = axes[2].set_ylabel('standardized residuals')
    _ = axes[2].set_title('normal qq plot')
    
    _ = axes[3].scatter(x=leverage, y=standardized_residual)
    _ = axes[3].set_xlabel('leverage')
    _ = axes[3].set_ylabel('standardized residuals')
    _ = axes[3].set_title(f'standardized residuals vs leverage')